
# Optimizando código Python

Como ya hemos mencionado **Python** es un lenguaje versátil pero poco eficiente en comparación a lenguajes de bajo nivel (C o Fortran)

Recordemos que **Python** es un lenguaje **interpretado**. Consideremos la operación

    z = x + y
    
Esta operación sencilla requiere de 

- **inferir el tipo** de $x$ e $y$ antes de sumarlos
- **escoger la función "suma" apropiada** 
- **retornar el tipo correcto** de z

El costo de las operaciones **destacadas** se llama **overhead**. El *overhead* es el **costo extra** de Python versus los lenguajes compilados

Existen varias maneras de reducir *overhead* y mejorar la eficiencia de un código escrito en Python, algunas de ellas son:

- **Conocer el lenguaje**: Usar la sintaxis y estructuras de Python adecuadamente 
- **Vectorización:** Cómputo basado en arreglos con `NumPy` siempre en problemas que lo permitan
- **Conectar con lenguajes de bajo nivel:** Utilizar `Cython` para crear interfaces de código C eficiente para Python


In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from functools import partial

## Conocer el lenguaje para ganar eficiencia

**Python** tiene una curva de aprendizaje suave en comparación a lenguajes de más bajo nivel, es decir que sabiendo muy poco de **Python** ya somos capaces de escribir toda clase de rutinas

Esto tiene un efecto secundario negativo en algunas personas y especialmente en aquellos que ya saben otros lenguajes 

:::{error}

Usar Python como si fuera C (u otro lenguaje)

:::

Python ofrece una gran cantidad de [funciones](https://docs.python.org/3/library/functions.html) y [módulos en su librería estándar](https://docs.python.org/3/library/index.html) que son sumamente eficientes

:::{hint}

Usar la sintáxis y las [estructuras de datos](https://docs.python.org/3/tutorial/datastructures.html) de Python adecuadamente es el primer paso para escribir código eficiente. Busque en la documentación de Python las estructuras de datos más apropiadas para cada problema

:::

Si necesita repasar sobre algoritmos se recomienda el siguiente material

- Bibliografía complementaria del curso: [Effective Python](https://effectivepython.com/)
- [Tratado de algoritmos y estructuras de datos en Python](https://runestone.academy/runestone/books/published/pythonds/index.html)
- [Consejos de velocidad en la Python wiki](https://wiki.python.org/moin/PythonSpeed)
- [Complejidad temporal de distintas estructuras de Python](https://wiki.python.org/moin/TimeComplexity)

A continuación veremos algunos consejos generales enfocados a Python

###  Evita usar `for` siempre que se pueda en favor de las funciones nativas

Muchas veces podemos evitar usar `for` con la estructura de datos o función adecuada

Para ejemplificar digamos que queremos sumar los valores absolutos de los elementos de una lista

In [2]:
x = [x for x in range(100000)]

# Suma estilo C 
def suma_abs(data):
    resultado = 0
    for i in range(len(data)):
        if data[i] > 0:
            resultado += data[i]
        else:
            resultado -= data[i]
    return resultado

%timeit -r5 -n3 suma_abs(x)
suma_abs(x)

15.8 ms ± 214 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)


4999950000

In [3]:
# Mejora 1: No es necesario usar un índice, podemos iterar directamente en los elementos
def suma_abs(data):
    resultado = 0
    for element in data:
        if element > 0:
            resultado += element
        else:
            resultado -= element
    return resultado

%timeit -r5 -n3 suma_abs(x)
suma_abs(x)

9.72 ms ± 661 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)


4999950000

In [4]:
# Mejora 2: Operar como una comprensión de lista y luego usar la función sum de Python
%timeit -r5 -n3 sum([x if x> 0 else -x for x in x])
sum([x if x> 0 else -x for x in x])

6.44 ms ± 857 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)


4999950000

In [5]:
# Mejora 3: Usar las funciones sum, map y abs de Python
%timeit -r5 -n3 sum(list(map(abs, x)))
sum(list(map(abs, x)))

3.39 ms ± 122 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)


4999950000

### No reinventes la rueda con las estructuras de datos

Verifica si la estructura que necesitas está implementada en Python antes de programarla tu

Como ejemplo digamos que queremos contar la cantidad de elementos de cada tipo en una lista. Podríamos escribir un contador como

In [6]:
x2 = list(np.random.randint(10, size=10000))

# Un contador de elementos
def miCounter(data):
    count = {}
    for element in data:
        if element not in count:
            count[element] = 1
        else:
            count[element] +=1
    return count

%timeit -r7 -n1 miCounter(x2)
miCounter(x2)

3.28 ms ± 169 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


{8: 1031,
 4: 980,
 9: 978,
 1: 985,
 2: 1011,
 3: 1009,
 6: 972,
 7: 1030,
 5: 987,
 0: 1017}

Sin embargo, la clase contador ya existe en `collections`, y es mucho más eficiente que la implementación "a mano"

In [7]:
from collections import Counter
%timeit -r7 -n1 Counter(x2)
Counter(x2)

1.82 ms ± 210 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Counter({8: 1031,
         4: 980,
         9: 978,
         1: 985,
         2: 1011,
         3: 1009,
         6: 972,
         7: 1030,
         5: 987,
         0: 1017})

### Ten atención con el overhead en funciones

Python tiene un overhead considerable cada vez que se llama una función

Se puede ganar desempeño haciendo *inlining* de funciones

In [8]:
def cuadradomasuno(element):
    return element*element + 1

%timeit -r7 -n3 [cuadradomasuno(xi) for xi in x]
#Inlining: escribo la función textualmente en lugar de evaluarla
%timeit -r7 -n3 [xi*xi + 1 for xi in x] 
np.allclose([cuadradomasuno(xi) for xi in x], [xi*xi + 1 for xi in x] )

23.4 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)
15.6 ms ± 440 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)


True

### Usa variables locales dentro de los *loops*

Si estamos obligados a usar `for` podemos ganar algo de rendimiento haciendo copias locales de atributos y funciones

Por ejemplo, digamos que queremos crear una lista con todos los elementos de otra lista que cumplen la condición

$$
\sin(x[i]) > 0 
$$


In [9]:
import math

# iterando sobre la lista
def sin_pos(data):
    resultado = []
    for element in data:
        if math.sin(element) > 0:
            resultado.append(element)
    return resultado

%timeit -r5 -n3 sin_pos(x)
resultado1 = sin_pos(x)

26.4 ms ± 1.73 ms per loop (mean ± std. dev. of 5 runs, 3 loops each)


Así se vería el código si hacemos variables locales para el método `append` y la función `sin`

In [10]:
# Mejora: variables locales
def sin_pos(data):
    resultado = []
    append = resultado.append
    sin = math.sin
    for element in data:
        if sin(element) > 0:
            append(element)
    return resultado

%timeit -r5 -n3 sin_pos(x)
resultado2 = sin_pos(x)
display(np.allclose(resultado1, resultado2))

15.6 ms ± 540 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)


True

## Vectorización: Cómputo basado en arreglos con NumPy

Consideremos el escenario en que tenemos un arreglo de datos de gran tamaño y queremos hacer una operación sobre cada elemento

**Ejemplo:** Dado $\{x\}_i$ queremos encontrar

$$
y_i = \frac{1}{1 + e^{-x_i}}, \quad i = 1,2,\ldots, N
$$

Los cómputos de este tipo se catalogan como *SIMD*: Single Instruction Multiple Data, es decir que estamos haciendo una misma operación para todos los datos

Este problema se puede resolver usando un ciclo `for` 

```python    
    y = []
    for xi in x:
        y.append(1/(1+math.exp(-xi))
```

Sin embargo, ya sabemos que esto es ineficiente en "Python puro"

No olvidemos que las librerías de cómputo científico vistas en este curso nos ofrecen una mejor alternativa. En particular, **NumPy** nos provee de una estructura de arreglo multidimensional (ndarray) y funciones para operarla que están escritas en C y Fortran. Otras librerías como **Scipy** y **Pandas** se basan en **NumPy**

> Cuando utilizamos estas librerías estamos usando código compilado en lugar de interpretado 

Usando **NumPy** podemos reemplazar un ciclo `for` en problemas *SIMD* por operaciones que trabajan sobre todo el arreglo, estas sa llaman **operaciones vectoriales**

Por ejemplo el código que vimos anteriormente quedaría como (asumiendo que x es una lista)

```python
    x = np.array(x)
    y = 1/(1+np.exp(-x))
```
    
NumPy aplica la función exponencial a todo el arreglo x, luego aplica la aritmética (suma y división) a cada elemento del arreglo (broadcasting) y finalmente retorna un nuevo arreglo con el resultado. En general siempre que exista una operación de tipo *SIMD* podemos mejorar su rendimiento usando 

> **Vectorización:** Reemplazar un bucle/ciclo por operaciones vectoriales de **NumPy**

A continuación revisaremos mediante ejemplos como implementar este y otros conceptos para mejorar el rendimiento

### Reemplazar ciclo `for` por operaciones vectoriales

Las operaciones vectoriales son **funciones de NumPy** de tipo *element-wise* aplicadas sobre un ndarray

Las funciones *element-wise* son aquellas que actuan sobre todos los elementos del arreglo de forma independiente

- En la clase de NumPy revisamos algunos ejemplos: exponenciación, raíces, trigonometrícas, etc
- Las operaciones aritméticas entre ndarrays son por defecto *element-wise*

Luego si tenemos un problema [SIMD](https://en.wikipedia.org/wiki/Single_instruction,_multiple_data) escrito con un `for` sobre un conjunto de datos podemos 

1. Convertir los datos a ndarray
1. Escribir la operación con funciones de NumPy *element-wise*

y ganar considerablemente en eficiencia de forma directa. Tenga presente que cuando las operaciones de NumPy se ejecutan sobre un ndarray se está usando código compilado

Por ejemplo notemos la diferencia en tiempo de cómputo al hacer aritmética simple

In [11]:
x_ndarray = np.random.randn(100000)
x_list = list(x_ndarray)

def operacion_simple(data):
    resultado = []
    append = resultado.append
    for elemento in data:
        append(elemento*elemento + elemento)
    return resultado

# Operación usando "for con mejoras"
%timeit -n3 -r7 operacion_simple(x_list)
# Operación usando numpy sobre un ndarray
%timeit -n3 -r7 x_ndarray*x_ndarray + x_ndarray
# Comparación entre los resultados
np.allclose(operacion_simple(x_list), x_ndarray*x_ndarray + x_ndarray)

23.6 ms ± 1.92 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)
197 µs ± 43.4 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)


True

:::{note}

Las funciones de NumPy son lentas cuando operan sobre tipos que no son ndarray

:::

Para el ejemplo de $y_i = (1 + e^{-x_i})^{-1}, \quad i = 1,2,\ldots, N$

In [12]:
from math import exp
# usando list comprehension (similar a un "for mejorado")
%timeit -n3 -r7 [1./(1.+exp(xi)) for xi in x_list]
# usando numpy sobre una lista
%timeit -n3 -r7 1./(1+np.exp(x_list))
# usando numpy sobre un ndarray
%timeit -n3 -r7 1./(1+np.exp(x_ndarray))
# Comparación entre los resultados
np.allclose(1./(1+np.exp(x_ndarray)), [1./(1.+exp(xi)) for xi in x_list])

18.2 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)
7.3 ms ± 66 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)
560 µs ± 37.6 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)


True

Las operación sobre ndarray es casi un orden de magnitud más rápida

**¿Cómo se explica esto?**

Recuerde que una lista puede tener distintos tipos y estar guardada en distintos sectores de memoria. En cambio, el ndarray

- Tiene un tipo definido 
- Está guardado en bloques de memoria contiguos

Por ende el ndarray tiene un overhead de interpretador mucho menor

Además NumPy está escrito en C/Fortran, hacer un loop en memoría contigua en C es muy eficiente

:::{note}

NumPy puede compilarse con librerías de alto desempeño (openblas, MKL) aprovechando mejor las capacidades del hardware (Cache de CPU e instrucciones vectoriales de CPU)

:::


### Convertir operaciones lógicas sobre arreglos en máscaras

Las operaciones lógicas en NumPy también son *element-wise* (Operaciones booleanas, clase NumPy, unidad 1)

Si queremos recuperar los elementos de un arreglo que cumplan una cierta condición podemos

1. Convertir los datos a ndarray
1. Escribir la operación como una máscara booleana de índices

Para el ejemplo anterior de recuperamos los elementos tal que $\sin(x_i)>0$

In [13]:
import math

def sin_pos(data):
    resultado = []
    append = resultado.append
    sin = math.sin
    for element in data:
        if sin(element) > 0:
            append(element)
    return resultado

%timeit -r5 -n3 sin_pos(x_list)

%timeit -r5 -n3 x_ndarray[np.sin(x_ndarray) > 0.]

display(np.allclose(sin_pos(x_list), x_ndarray[np.sin(x_ndarray) > 0]))

15.4 ms ± 1.09 ms per loop (mean ± std. dev. of 5 runs, 3 loops each)
3.79 ms ± 30.9 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)


True

### Usa operaciónes *in-place* y vistas de arreglos para evitar copia extra de datos

Cuando en NumPy hacemos

```python
    x = x*x
```

1. Se crea una copia interna de x*x
1. x es direccionado a esa nueva copia
1. La zona de memoria con el valor original es luego eliminada por el *garbage-collector* de Python

Siempre que no necesitemos el valor original podemos usar operaciones *in-place* y ganar rendimiento, ya que evitamos la copia y eliminación adicional

In [14]:
# Copia interna y cambio de referencia de x_ndarray 
%timeit -r10 -n10 x_ndarray = np.zeros(shape=(1000000)); y = x_ndarray*x_ndarray
# Sin copia interna
%timeit -r10 -n10 x_ndarray = np.zeros(shape=(1000000)); x_ndarray *= x_ndarray
# El resultado es idéntico
x_ndarray = np.zeros(shape=(1000000))
y = x_ndarray*x_ndarray
x_ndarray *= x_ndarray
np.allclose(x_ndarray, y)

3.54 ms ± 44.2 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
2.69 ms ± 79.3 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


True

Sea $x$ un ndarray, la operación

```python
    x[2:10] 
```

es una "vista de x". 

:::{hint}

Recordar que las "vista de arreglo" no hacen copias en memoria ya que apuntan directamente al arreglo original. Es decir que si modificamos una vista modificamos el original

:::

### Aprovecha el *broadcasting* automático de NumPy

Se pueden hacer operaciones vectorizadas con NumPy entre arreglos con tamaños distintos. En ese caso se aplican las reglas de *broadcasting* que vimos en la clase de NumPy

:::{note}

El *broadcasting* automático no hace copias en memoria

:::

**Ejemplo 1:** Si le sumas una constante a un arreglo 1D, la constante se expande y se suma a cada elemento

In [15]:
N = 1000000
x = np.zeros(shape=(N, ))
# broadcasting automático
%timeit -n10 -r10 x + 1.
# Agrandando y luego sumando
%timeit -n10 -r10 x + np.tile([1], len(x))
# mismo resultado
np.allclose(x + 1, x + np.tile([1], len(x)))

638 µs ± 220 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
10.4 ms ± 375 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


True

**Ejemplo 2:** Si le sumas un arreglo 1D a un arreglo 2D, el arreglo 1D se expande en la dimensión que le falta

In [16]:
N, M = 10000, 1000
x = np.zeros(shape=(N, M)) # arreglo de NxM
y = np.zeros(shape=(N, )) # arreglo sin dimensión
y_ = y[:, np.newaxis] # arreglo de Nx1
# broadcasting automático
%timeit -n10 -r10 x + y_
display((x + y_).shape)
# Agrandando y luego sumando
%timeit -n10 -r10 x + np.tile(y_, (1, x.shape[-1]))
# mismo resultado
np.allclose(x + y_, x + np.tile(y_, (1, x.shape[-1])))

30.3 ms ± 959 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


(10000, 1000)

86.5 ms ± 1.87 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


True

**Ejemplo 3:** Si sumas un arreglo 1D fila y un arreglo 1D columna se crea un arreglo 2D

In [17]:
N, M = 10000, 1000
x = np.zeros(shape=(N, 1)) # arreglo columna de Nx1
y = np.zeros(shape=(1, M)) # arreglo fila de 1xM
# broadcasting automático
%timeit -n10 -r10 x + y
display((x+y).shape)
# Agrandando y luego sumando
%timeit -n10 -r10 np.tile(y, (x.shape[0], 1)) + np.tile(x, (1, y.shape[-1]))
np.allclose(x + y, np.tile(y, (x.shape[0], 1)) + np.tile(x, (1, y.shape[-1])))

35.2 ms ± 859 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


(10000, 1000)

114 ms ± 1.76 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


True

:::{note}

Las dimensiones de dos arreglos son compatibles con *broadcast* automático si **son del mismo tamaño** o **una de ellas es igual a uno**

:::

### Utiliza el ordenamiento en memoría más adecuado en cada caso

Los ndarray multidimensionales pueden guardarse en memoria como *row-major* (filas contiguas) o *column-major* (columnas contiguas)

Por defecto las matrices en NumPy son *row-major* pero podemos forzar la contigüidad usando el atributo `order` o trasponiendo (ojo que trasponer crea una copia)

Se puede verificar esto con el atributo `flag` de los ndarray

In [18]:
data = np.arange(6).reshape(2, 3)
display(data)
# Verificamos los flags
display(data.flags)
# Así se ve row-major en memoria 
display(data.ravel())
# Verificamos los flags
dataT = data.T
display(dataT.flags)
# Así se ve column-major en memoria
display(dataT.ravel())

array([[0, 1, 2],
       [3, 4, 5]])

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

array([0, 1, 2, 3, 4, 5])

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

array([0, 3, 1, 4, 2, 5])

La mayoría de las funciones de NumPy funcionan más rápido en formato *row-major* (formato C). Pero algunas funciones de scipy (heredadas de Fortran) funcionan más rápido en formato *col-major* (formato Fortran)

Es recomendable verificar el orden en memoria que espera la función que vas a utilizar

In [19]:
data = np.random.randn(10000, 10000) # (row-major)
# Sumando una fila
%timeit -n100 -r10 np.sum(data[0, :])
# Sumando todas las filas
%timeit -n10 -r10 np.sum(data, axis=1)

12.9 µs ± 405 ns per loop (mean ± std. dev. of 10 runs, 100 loops each)
73.5 ms ± 959 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [20]:
# Sumando una columna
%timeit -n100 -r10 np.sum(data[:, 0])
# Sumando todas las columnas
%timeit -n10 -r10 np.sum(data, axis=0)

31 µs ± 6.24 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
100 ms ± 1 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [21]:
# Sumando todas las columnas de la matriz traspuesta (column major)
%timeit -n10 -r10 np.sum(data.T, axis=0)

72.3 ms ± 552 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


## Speed-up



El ***[speedup](https://en.wikipedia.org/wiki/Speedup)*** es un número que mide el desempeño relativo de dos algoritmos, sistemas o rutinas.

Para este caso podemos escribir
$$
S_{tiempo} = \frac{T_{referencia}}{T_{propuesto}}
$$
donde $T_{propuesto}$ y $T_{referencia}$ son los tiempos de cómputo de nuestra rutina propuesta y de la rutina de referencia, respectivamente

Este speedup temporal indica cuantas veces más rápido es nuestra rutina propuesta con respecto a la referencia